# Build a content-based file tagging system with LLMs on Amazon Bedrock

[Amazon S3](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html) is a popular object storage service on AWS. You can store any type of file as an object in a S3 bucket. Although you can write files of a specific type or context within a specific directory structure (path) in S3, it will be useful to add metadata to the files like it's content description, owner, context etc. so you can easily retrieve the file that you are looking for. There are two ways to do this:

**Option 1: Use the user-defined metadata feature in S3**

While uploading an object to a S3 bucket, you can optionally assign user-defined metadata as key-values pairs to the object. This will be stored along with the object. This cannot be added later on to an existing object. The only way to modify object metadata is to make a copy of the object and set the metadata.

**Option 2: Store the metadata in an external system with a reference to the object in S3**

If you want to set metadata to an existing object in S3 without copying that object or if you want to add to an existing metadata system that already exist, then it will make sense to store the metadata in an external system, like an Amazon DynamoDB table for example. This option is also applicable if the data is stored outside S3 and needs to be tagged with metadata.

In both of these options, if you do not know the metadata that describes the data stored in the object, then, you have to read the object, analyze it's content and generate the appropriate metadata. This is where AI can help.

[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing Foundation Models (FMs) from leading AI companies accessible through a single API, along with a broad set of capabilities you need to build generative AI applications, simplifying development while maintaining privacy and security.

[Large Language Models (LLMs)](https://en.wikipedia.org/wiki/Large_language_model) are a type of Foundation Model that can take natural langauge as input, with the ability to process and understand it, and produce natural language as the output. LLMs can also can perform tasks like classification, summarization, simplification, entity recognition, etc.

This notebook will walk you through the process of analyzing text-content or image-content in files stored in S3 using a Large Language Model (LLM) hosted on [Amazon Bedrock](https://aws.amazon.com/bedrock/), generating metadata based on the content of the files and storing them as key-value pairs (tags) in an [Amazon DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Introduction.html) table with a reference to the files in S3. In the process, you will learn how to setup the Amazon Bedrock client environment, configure security permissions and use prompt templates. You will also learn how to use Amazon Bedrock's [Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) to consistently interact with multiple supported LLMs and easily send documents of various types into the prompts of those LLMs. Finally, you will learn some techniques for analyzing files with mixed content (text and images).

<div class="alert alert-block alert-info">
<b>Note:</b>
    <ul>
        <li>This notebook should only be run from within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html">Amazon SageMaker Notebook instance</a> or within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated.html">Amazon SageMaker Studio Notebook</a>.</li>
        <li>At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions that has <b>Amazon Nova 1.0 (Lite/Pro) or Anthropic Claude 3 (Haiku/Sonnet/Opus) or Anthropic Claude 3.5 (Sonnet v2/Sonnet) or Meta Llama 3.2 (11B/90B Instruct)</b>. If available, you can use <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html">cross-region inference</a>. Follow the guidance in the <i>Organize imports</i> section of this notebook.</li>
        <li>Amazon Bedrock Converse API supports <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-supported-models-features.html">these models</a>. If you intend to use any Converse API supported model other than the ones used in this notebook, make sure both text and image (vision) are supported as input modalities.</li>
        <li>At the time of writing this notebook, Converse API supported:
            <ul>
                <li>Only these document file types -> <b>pdf, csv, doc, docx, xls, xlsx, html, txt, md</b> with up to <b>5 documents</b> in one API call. The max size of a file can be <b>4.5 MB</b>. So make sure your files are one of these types with the right file extension and within the max size limit.</li>
                <li>Only these image file types -> <b>png, jpeg, gif, webp</b> with up to <b>20 images</b> in one API call. Each image’s size, height, and width must be no more than <b>3.75 MB, 8000 px, and 8000 px,</b> respectively. So make sure your files are one of these types with the right file extension and within the max size limit.</li>
            </ul>
        </li>
        <li>This notebook expects files to be readily available in the specified S3 bucket.</li>
        <li>This notebook will process <b>either text or image content</b>. If a file contains mixed content (text and image), then, only the text part of the content will be processed; the image part of the content will be ignored.</li>
        <li>This notebook is recommended to be run with a minimum instance size of <i>ml.t3.medium</i> and
            <ul>
                <li>With <i>Amazon Linux 2, Jupyter Lab 4</i> as the platform identifier on an Amazon SageMaker Notebook instance.</li>
                <li> (or)
                <li>With <i>Data Science 3.0</i> as the image on an Amazon SageMaker Studio Notebook.</li>
            <ul>
        </li>
        <li>At the time of this writing, the most relevant latest version of the Kernel for running this notebook,
            <ul>
                <li>On an Amazon SageMaker Notebook instance was <i>conda_python3</i></li>
                <li>On an Amazon SageMaker Studio Notebook was <i>Python 3</i></li>
            </ul>
        </li>
    </ul>
</div>

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)

    1. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    2. [Install required software libraries](#Install%20required%20software%20libraries)
    
    3. [Configure logging](#Configure%20logging)
        
        1. [System logs (Optional)](#Configure%20system%20logs%20(Optional))
        
        2. [Application logs](#Configure%20application%20logs)
    
    4. [Organize imports](#Organize%20imports)
    
    5. [Set AWS Region and boto3 config](#Set%20AWS%20Region%20and%20boto3%20config)
    
    6. [Enable model access in Amazon Bedrock](#Enable%20model%20access%20in%20Amazon%20Bedrock)
    
    7. [Create common objects](#Create%20common%20objects)
    
    8. [Check and create the metadata table](#Check%20and%20create%20the%20metadata%20table)
    
    9. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)

 2. [Download files from S3](#Download%20files%20from%20S3)
 
 3. [Generate and store the file tags in the metadata table](#Generate%20and%20store%20the%20file%20tags%20in%20the%20metadata%20table)
 
 4. [Retrieve the file tags from the metadata table](#Retrieve%20the%20file%20tags%20from%20the%20metadata%20table)
 
 5. [How to analyze mixed content?](#How%20to%20analyze%20mixed%20content)
 
 6. [Cleanup](#Cleanup)
 
 7. [Conclusion](#Conclusion)
 
 8. [Frequently Asked Questions (FAQs)](#FAQs)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

Check and complete the prerequisites.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>
This notebook requires outbound access to the Internet to download the required software updates and to download the dataset.  You can either provide direct Internet access (default) or provide Internet access through an [Amazon VPC](https://aws.amazon.com/vpc/).  For more information on this, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html).

### B. Install required software libraries <a id ='Install%20required%20software%20libraries'> </a>
This notebook requires the following libraries:
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [Python 3.10.x](https://www.python.org/downloads/release/python-3100/)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)

Run the following cell to install the required libraries.

<div class="alert alert-block alert-warning">  
    <b>Note:</b> At the end of the installation, the Kernel will be forcefully restarted immediately. Please wait 10 seconds for the kernel to come back before running the next cell.
</div>

In [ ]:
!pip install boto3==1.35.94
!pip install sagemaker==2.237.1

import IPython

IPython.Application.instance().kernel.do_shutdown(True)

### C. Configure logging <a id ='Configure%20logging'> </a>

####  a. System logs (Optional) <a id='Configure%20system%20logs%20(Optional)'></a>

System logs refers to the logs generated by the notebook's interactions with the underlying notebook instance. Some examples of these are the logs generated when loading or saving the notebook.

These logs are automatically setup when the notebook instance is launched.

These logs can be accessed through the [Amazon CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html) console in the same AWS Region where this notebook is running.
* When running this notebook in an Amazon SageMaker Notebook instance, navigate to the following location,
    * <i>CloudWatch > Log groups > /aws/sagemaker/NotebookInstances > {notebook-instance-name}/jupyter.log</i>
* When running this notebook in an Amazon SageMaker Studio Notebook, navigate to the following locations,
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/KernelGateway/{notebook-instance-name}</i>
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/JupyterServer/default</i>

If you want to find out the name of the underlying instance where this notebook is running, uncomment the following code cell and run it.

In [ ]:
'''
import json

notebook_name = ''
resource_metadata_path = '/opt/ml/metadata/resource-metadata.json'
with open(resource_metadata_path, 'r') as metadata:
    notebook_name = (json.load(metadata))['ResourceName']
print("Notebook instance name: '{}'".format(notebook_name))
'''

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  D. Organize imports <a id ='Organize%20imports'> </a>

Organize all the library and module imports for later use.

In [ ]:
import boto3
import sagemaker
import sys
from botocore.config import Config

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

Print the installed versions of some of the important libraries.

In [ ]:
logging.info("Python version : {}".format(sys.version))
logging.info("Boto3 version : {}".format(boto3.__version__))
logging.info("SageMaker Python SDK version : {}".format(sagemaker.__version__))

###  E. Set AWS Region and boto3 config <a id ='Set%20AWS%20Region%20and%20boto3%20config'> </a>

Get the current AWS Region (where this notebook is running) and the SageMaker Session. These will be used to initialize some of the clients to AWS services using the boto3 APIs.

<div class="alert alert-block alert-info">  
<b>Note:</b> All the AWS services used by this notebook except Amazon Bedrock will use the current AWS Region. For Bedrock, follow the guidance in the next cell.
</div>

<div class="alert alert-block alert-warning">  
<b>Note:</b> At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions that has Amazon Nova 1.0 (Lite/Pro) or Anthropic Claude 3 (Haiku/Sonnet/Opus) or Anthropic Claude 3.5 (Sonnet v2/Sonnet) or Meta Llama 3.2 (11B/90B Instruct). In order to do this, this notebook will use the value specified in the environment variable named <mark>AMAZON_BEDROCK_REGION</mark>. If this is not specified, then the notebook will default to <mark>us-west-2 (Oregon)</mark> for Amazon Bedrock. If available, you can use <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html">cross-region inference</a>.
</div>



In [ ]:
# Get the AWS Region, SageMaker Session and IAM Role references
my_session = boto3.session.Session()
logging.info("SageMaker Session: {}".format(my_session))
my_iam_role = sagemaker.get_execution_role()
logging.info("Notebook IAM Role: {}".format(my_iam_role))
my_region = my_session.region_name
logging.info("Current AWS Region: {}".format(my_region))

# Explicity set the AWS Region for Amazon Bedrock clients
AMAZON_BEDROCK_DEFAULT_REGION = "us-west-2"
br_region = os.environ.get('AMAZON_BEDROCK_REGION')
if br_region is None:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
elif len(br_region) == 0:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
logging.info("AWS Region for Amazon Bedrock: {}".format(br_region))

Set the timeout and retry configurations that will be applied to all the boto3 clients used in this notebook.

In [ ]:
# Increase the standard time out limits in the boto3 client from 1 minute to 3 minutes
# and set the retry limits
my_boto3_config = Config(
    connect_timeout = (60 * 3),
    read_timeout = (60 * 3),
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

###  F. Enable model access in Amazon Bedrock <a id ='Enable%20model%20access%20in%20Amazon%20Bedrock'> </a>

<div class="alert alert-block alert-danger">
    <b>Note:</b> Before proceeding further with this notebook, you must enable access to the required models on Amazon Bedrock by following the instructions <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">here</a>. Otherwise, you will get an authorization error.
</div>

<div class="alert alert-block alert-warning">  
<b>Note:</b> You will have to do this manually after reading the End User License Agreement (EULA) for each of the models that you want to enable. Unless you explicitly disable it, this is a one-time setup for each model in an AWS Region in an AWS account.
</div>

Run the following cell to print the Amazon Bedrock model access page URL for the AWS Region that was selected earlier.

In [ ]:
# Print the Amazon Bedrock model access page URL
logging.info("Amazon Bedrock model access page - https://{}.console.aws.amazon.com/bedrock/home?region={}#/modelaccess"
             .format(br_region, br_region))

###  G. Create common objects <a id='Create%20common%20objects'></a>

To begin with, create the boto3 clients.

In [ ]:
# Create the Amazon Bedrock client
bedrock_client = boto3.client("bedrock", region_name = br_region, endpoint_url = "https://bedrock.{}.amazonaws.com"
                              .format(br_region), config = my_boto3_config)

# Create the Amazon Bedrock runtime client
bedrock_rt_client = boto3.client("bedrock-runtime", region_name = br_region, config = my_boto3_config)

In the below cell, modify the value of `model_or_inference profile_id` as needed. For information on retrieving the right model or inference profile id, perform the following.

1. Look at the list of [models supported by Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-supported-models-features.html). From this, pick the one that support both text and image (vision) as it's input.
2. For the model you picked,
    - Get the on-demand throughput model id from [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns). While running the below cell with that model id, if you get an error that states it is not supported for on-demand throughput, then, proceed to the next step to use the inference profile id.
    - Get the inference profile id by following the procedure mentioned [here](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-view.html).

In [ ]:
# Specify the model id (for Om-Demand) or inference profile id (for Inference Profile)
#model_or_inference_profile_id = "us.amazon.nova-lite-v1:0"
#model_or_inference_profile_id = "us.amazon.nova-pro-v1:0"
model_or_inference_profile_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
#model_or_inference_profile_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
#model_or_inference_profile_id = "anthropic.claude-3-haiku-20240307-v1:0"
#model_or_inference_profile_id = "anthropic.claude-3-sonnet-20240229-v1:0"
#model_or_inference_profile_id = "anthropic.claude-3-opus-20240229-v1:0"
#model_or_inference_profile_id = "us.meta.llama3-2-11b-instruct-v1:0"
#model_or_inference_profile_id = "us.meta.llama3-2-90b-instruct-v1:0"

# Specify the Amazon S3 bucket name that will contain all the source files
s3_bucket_name = "<The name of the input S3 bucket>"

# Specify the S3 key prefix to refer to one or more files in the bucket
# Note: If the key prefix is empty, then, all the source files in the bucket
# will be downloaded with the exact directory structure as in the bucket.
s3_key_prefix = ""

# Specify the name of the DynamoDB table that will contain the metadata
ddb_table_name = "<The name of the DynamoDB table that will store the metadata>"

# Specify the name and location of the prompt templates
prompt_templates_dir = os.path.join(os.getcwd(), "prompt_templates")
system_prompt_template_file = 'system_prompt_template.txt'
user_prompt_template_file = 'user_prompt_template.txt'

# Specify and create the required output directories
data_dir = os.path.join(os.getcwd(), "data")
os.makedirs(data_dir, exist_ok = True)

###  H. Check and create the metadata table <a id ='Check%20and%20create%20the%20metadata%20table'> </a>

Run the following cell to check and create the specified Amazon DynamoDB table in the current AWS Region. This table will contain the metadata for the objects stored in S3.

In [ ]:
# Create the Amazon DynamoDB clients
ddb_client = boto3.client("dynamodb", region_name = my_region, config = my_boto3_config)
ddb_resource = boto3.resource("dynamodb", region_name = my_region, config = my_boto3_config)

# Note: 'check_and_create_metadata_table' is available through ./scripts/helper_functions.py
check_and_create_metadata_table(ddb_client, ddb_table_name)

###  I. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
This notebook uses the IAM role attached to the underlying notebook instance.  To view the name of this role, run the following cell. This IAM role should have the following permissions,
1. Full access to invoke Large Language Models (LLMs) on Amazon Bedrock.
2. Full access to read and write to the Amazon S3 bucket that contains the files to be tagged.
3. Full access to read and write to the Amazon DynamoDB table that will contain the metadata.
4. Access to write to Amazon CloudWatch Logs.

<div class="alert alert-block alert-info">
<b>Note:</b>  If you are running this notebook as part of a workshop session, by default, all these permissions will be setup.
</div>

Run the following cell to print the details of the IAM role attached to the underlying notebook instance.

In [ ]:
# Print the IAM role ARN and console URL
logging.info("This notebook's IAM role is '{}'".format(my_iam_role))
arn_parts = my_iam_role.split('/')
logging.info("Details of this IAM role are available at https://{}.console.aws.amazon.com/iamv2/home?region={}#/roles/details/{}?section=permissions"
             .format(my_region, my_region, arn_parts[len(arn_parts) - 1]))

## 2. Download files from S3 <a id ='Download%20files%20from%20S3'> </a>

This notebook expects files to be readily available in the specified S3 bucket.

<div class="alert alert-block alert-info">
<b>Note:</b> At the time of writing this notebook, Converse API supported:
    <ul>
        <li>Only these document file types -> <b>pdf, csv, doc, docx, xls, xlsx, html, txt, md</b> with up to <b>5 documents</b> in one API call. The max size of a file can be <b>4.5 MB</b>. So make sure your files are one of these types with the right file extension and within the max size limit.</li>
        <li>Only these image file types -> <b>png, jpeg, gif, webp</b> with up to <b>20 images</b> in one API call. Each image's size, height, and width must be no more than <b>3.75 MB, 8000 px, and 8000 px,</b> respectively. So make sure your files are one of these types with the right file extension and within the max size limit.</li>
    </ul>
</div>

Running the following cell will download the files from the S3 bucket to the specified local directory. Existing files will be overwritten.

In [ ]:
# Note: 'download_from_s3' is available through ./scripts/helper_functions.py
downloaded_file_paths = download_from_s3(data_dir, s3_bucket_name, s3_key_prefix)
logging.info("Full path(s) of the download file(s): {}".format(downloaded_file_paths))

## 3. Generate and store the file tags in the metadata table <a id ='Generate%20and%20store%20the%20file%20tags%20in%20the%20metadata%20table'> </a>

Loop through all the downloaded files and process as follows:
1. Read the file content as bytes.
2. Feed the file content bytes along with specific instructions to the specified LLM on Amazon Bedrock through the [Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html). The instructions should ask the LLM to identify the topic of the content of the file along with the flag that indicates the presence of PII data. Also, specify to the LLM to produce the output as a valid JSON message.
3. Insert/Update the response from the LLM in the specified metadata table in DynamoDB using the file's S3 URI as the Partition Key.

<div class="alert alert-block alert-warning">
<b>Note:</b>
    <ul>
        <li>Unsupported file types will be igored.</li>
        <li>In document files, only text content will be processed. Image content, if any, will be ignored. If the file contains only image content, then, the document type will be returned as 'empty' or 'blank'.</li>
        <li>In image files, the image content will be processed.</li>
    </ul>
</div>

In [ ]:
# Initialize the list to capture the processed files
processed_s3_uris = []

# Loop through the downloaded files
for downloaded_file_path in downloaded_file_paths:
    # Check if the file type is supported by Converse API and process accordingly
    # Note: 'is_supported_file_type' is available through ./scripts/helper_functions.py
    if is_supported_file_type(downloaded_file_path):
        # Prompt the LLM
        # Note: 'process_prompt' is available through ./scripts/helper_functions.py
        file_metadata = process_prompt(model_or_inference_profile_id, bedrock_rt_client, prompt_templates_dir,
                                       system_prompt_template_file, user_prompt_template_file,
                                       downloaded_file_path)
        # Insert/update the metadata table
        file_s3_uri = 's3://' + downloaded_file_path.replace(data_dir, s3_bucket_name)
        # Note: 'write_to_metadata_table' is available through ./scripts/helper_functions.py
        write_to_metadata_table(ddb_resource, ddb_table_name, file_s3_uri, file_metadata)
        # Add to the processed file list
        processed_s3_uris.append(file_s3_uri)

## 4. Retrieve the file tags from the metadata table <a id='Retrieve%20the%20file%20tags%20from%20the%20metadata%20table'></a>

Loop through all the processed files, retrieve their corresponding file tags from the metadata table and print them.

In [ ]:
# Loop through the processed files
for processed_s3_uri in processed_s3_uris:
    # Retrieve the metadata
    # Note: 'retrieve_from_metadata_table' is available through ./scripts/helper_functions.py
    logging.info("File: {}, Metadata = {}".format(processed_s3_uri, retrieve_from_metadata_table(ddb_resource,
                                                                                                 ddb_table_name,
                                                                                                 processed_s3_uri)))   

## 5. How to analyze mixed content? <a id='How%20to%20analyze%20mixed%20content'></a>

Here are some techniques for analyzing mixed content (text and image) files. In all these, you will have to use the relevant third-party libraries to read the files of specific types like pdf, doc, docx, xls, xlsx etc.

**Technique 1**: Prior to invoking the LLM using the Amazon Bedrock Converse API, convert every page in the document to an image. This will be like taking a screenshot of every page in the document. When you do this, you will be dealing with only images and no text at all. Now, you can invoke the LLM using the Converse API by specifying the content type as 'image' and pass in the image bytes. Make sure you pick a LLM that supports [Vision with Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-supported-models-features.html). You can refer to the code sample [here](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-examples.html). At the time of writing this notebook, Converse API supported up to 20 images in one API call. Each image’s size, height, and width must be no more than 3.75 MB, 8000 px, and 8000 px, respectively.

**Technique 2**: From each page in a document, read the image content separately as bytes. Then, read the text content (if any) separately. Now, create a list of messages with these info and send them to the LLM in the prompt. For example, if you are using Anthropic Claude 3.x models on Amazon Bedrock that support image (vision) and text in the input, then, you can use the [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html#model-parameters-anthropic-claude-messages-request-response) for this. Note that, depending on the LLM-specific API, you may have to encode the bytes as Base64 and also specify the image type.

## 6. Cleanup <a id='Cleanup'></a>

As a best practice, you should delete AWS resources that are no longer required.  This will help you avoid incurring unncessary costs.

<div class="alert alert-block alert-info">
<b>Note:</b> If you are running this notebook as part of a workshop session, by default, all resources will be cleaned up at the end of the session. If you are running this notebook outside of a workshop session, you can cleanup the resources associated with this notebook by uncommenting the following code cell and running it.
</div>

Running the following cell will delete the following resources:
* The DynamoDB table that was created to store the metadata.
* The files that were uploaded to the S3 bucket; not the S3 bucket itself.
* The files that were downloaded from the S3 bucket to the local directory.

In [ ]:
'''
# Note: 'delete_metadata_table' is available through ./scripts/helper_functions.py
delete_metadata_table(ddb_client, ddb_table_name)

# Create the Amazon S3 client
s3_client = boto3.client("s3", region_name = my_region, config = my_boto3_config)
# Loop through the downloaded files
for downloaded_file_path in downloaded_file_paths:
    # Get the S3 key
    s3_key = downloaded_file_path.replace(data_dir + os.sep, '')
    # Note: 'delete_from_s3' is available through ./scripts/helper_functions.py
    delete_from_s3(s3_client, s3_bucket_name, s3_key)
    # Note: 'delete_local_file' is available through ./scripts/helper_functions.py
    delete_local_file(downloaded_file_path)
'''

## 7. Conclusion <a id='Conclusion'></a>

We have now seen how to build a content-based file tagging system using the LLMs on Amazon Bedrock for text-only or image-only content. In the process, we also saw how easy it is to consistently interact with multiple supported LLMs on Amazon Bedrock using the Converse API with the ability to send in documents of various types in the prompt. Finally, we learnt some techniques for analyzing files with mixed content (text and images).

## 8. Frequently Asked Questions (FAQs) <a id='FAQs'></a>

**Q: What AWS services are used in this notebook?**

Amazon Bedrock, Amazon DynamoDB, Amazon S3, AWS Identity and Access Management (IAM), Amazon CloudWatch, and Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook depending on what you use to run the notebook.

**Q: Where can I access the FAQs for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/faqs/).

**Q: What models are supported by Amazon Bedrock?**

Go [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html).

**Q: Where can I find customer references for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/testimonials/).

**Q: Where can I find resources for prompt engineering?**

[Prompt Engineering Guide](https://www.promptingguide.ai/).

**Q: Where can I find pricing information for the AWS services used in this notebook?**

- Amazon Bedrock pricing - go [here](https://aws.amazon.com/bedrock/pricing/).
- Amazon DynamoDB pricing - go [here](https://aws.amazon.com/dynamodb/pricing/).
- Amazon S3 pricing - go [here](https://aws.amazon.com/s3/pricing/).
- AWS Identity and Access Management (IAM) pricing - free.
- Amazon CloudWatch pricing - go [here](https://aws.amazon.com/cloudwatch/pricing/).
- Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook pricing - go [here](https://aws.amazon.com/sagemaker/pricing/).